In [1]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, clean_words, parse_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle 
from training import train, validate, evaluate, evaluate_better, make_embedding_matrix, make_embedding_matrix, evaluator, evaluator_ir
from training import get_optimizer

from supervised_models import BiLSTM_model

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [3]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [4]:
training_instances = load_dataset_from_pickle('../data/training.pickle')
validation_instances = load_dataset_from_pickle('../data/validation.pickle')
testing_instances = load_dataset_from_pickle('../data/testing.pickle')
mixed_training = load_dataset_from_pickle('../data/mixed_oversampling_training.pickle')

In [5]:
vectorizer = Vectorizer.vectorize_training(mixed_training)
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [6]:
trainset = HeadQA(instances=mixed_training, vectorizer=vectorizer, right_padding=False, max_length=30)
validset = HeadQA(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
testset = HeadQA(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=30)

In [7]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [8]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors.pickle')

In [9]:
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

In [10]:
torch.random.manual_seed(42)
model = BiLSTM_model(embedding_matrix.shape[1], embedding_matrix.shape[0], 1, 
                     pretrained_embeddings=embedding_matrix, max_length=trainset.max_length)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [11]:
training_results = train(model, optimizer, train_dt, valid_dt, validate, epochs=50)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 train loss  0.3673 valid loss 0.455 and accuracy 0.2500
Epoch 1 train loss  57.1644 valid loss 0.455 and accuracy 0.2500
Epoch 2 train loss  57.1644 valid loss 0.455 and accuracy 0.2500
Epoch 3 train loss  57.1644 valid loss 0.455 and accuracy 0.2500
Epoch 4 train loss  21.4802 valid loss 0.258 and accuracy 0.2500
Epoch 5 train loss  1.9695 valid loss 0.058 and accuracy 0.2500
Epoch 6 train loss  0.5318 valid loss 0.055 and accuracy 0.2500
Epoch 7 train loss  0.5895 valid loss 0.054 and accuracy 0.2500
Epoch 8 train loss  0.5624 valid loss 0.039 and accuracy 0.2500
Epoch 9 train loss  0.4863 valid loss 0.037 and accuracy 0.2500
Epoch 10 train loss  0.5024 valid loss 0.043 and accuracy 0.2500
Epoch 11 train loss  0.5171 valid loss 0.049 and accuracy 0.2500
Epoch 12 train loss  0.5272 valid loss 0.056 and accuracy 0.2500
Epoch 13 train loss  0.5570 valid loss 0.055 and accuracy 0.2500
Epoch 14 train loss  0.5365 valid loss 0.063 and accuracy 0.2500
Epoch 15 train loss  0.5758 val

In [12]:
acc, points = evaluate(model, validation, trainset.encode, evaluator)
acc, points

(tensor([0.2408]), -50)

In [13]:
acc, points = evaluate(model, testing, trainset.encode, evaluator)
acc, points

(tensor([0.2465]), -38)

In [14]:
save_dataset_to_pickle('results_v2/train_results_bilstm.pickle', training_results)
training_results = load_dataset_from_pickle('results_v2/train_results_bilstm.pickle')

In [15]:
model_path = os.getcwd() + '/trained_models_v2/bilstm'
torch.save(model.state_dict(), model_path)

In [16]:
model = BiLSTM_model(embedding_matrix.shape[1], embedding_matrix.shape[0], 1, 
                     pretrained_embeddings=embedding_matrix, max_length=trainset.max_length)
model.load_state_dict(torch.load(model_path))
model.eval()

BiLSTM_model(
  (emb): Embedding(28831, 300, padding_idx=0)
  (dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(300, 64, batch_first=True, dropout=0.5, bidirectional=True)
  (attn): Linear(in_features=128, out_features=30, bias=True)
  (linear): Linear(in_features=3840, out_features=1, bias=True)
)

In [17]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.24080546,
 -8.333333333333334,
 [tensor(0.2566),
  tensor(0.2217),
  tensor(0.2267),
  tensor(0.2597),
  tensor(0.2257),
  tensor(0.2544)],
 [6, -26, -21, 9, -22, 4])

In [18]:
acc, points, acc_list, points_list = evaluate_better(model, testing, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.24649875,
 -3.1666666666666665,
 [tensor(0.2544),
  tensor(0.2108),
  tensor(0.2939),
  tensor(0.2284),
  tensor(0.2652),
  tensor(0.2511),
  tensor(0.1903),
  tensor(0.2414),
  tensor(0.2620),
  tensor(0.2251),
  tensor(0.2756),
  tensor(0.2599)],
 [4, -35, 40, -20, 14, 1, -54, -8, 11, -23, 23, 9])